In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch_geometric.transforms as T

from models import TemporalGraphTrainer, GTCModel, Traj2VecModel

In [17]:
unmapped_traj = pd.read_csv("../../../datasets/trajectories/hanover/mapped_id_poly_clipped.csv", ";")
print(unmapped_traj[unmapped_traj["id"]==3810])
print("--------")
print(traj[traj["id"]==3810])

                                              timestamp  start_stamp  \
3809     0    3   33   38   48   49   53   63   65  ...   1574703171   

       end_stamp                                           POLYLINE    id  
3809  1574706343  LINESTRING (9.695580489293073 52.3383265887556...  3810  
--------
     id                   seg_seq  travel_time
0  3810  [1575, 7994, 6088, 1283]           12


In [9]:
# Generate traj-adj matrix for gtc
# 1. Load required data
traj = Trajectory("../../../datasets/trajectories/hanover/temporal/road_segment_map_final.csv", nrows=100000000).generate_TTE_datatset()
traj["seg_seq"] = traj["seg_seq"].map(np.array)
network = RoadNetwork()
network.load_hanover_temporal(path="../../../datasets/trajectories/hanover/temporal/hannover_streetgraph.csv")

Pandas Apply:   0%|          | 0/57996 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/57996 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/57996 [00:00<?, ?it/s]

       Unnamed: 0      id                                              opath  \
0              59    3810   [1575, 1575, 1575, 6088, 1283, 1283, 1283, 1283]   
1             226    1872  [4688, 4688, 6882, 6882, 6882, 6882, 6882, 688...   
2             267    3814  [866, 7513, 7513, 7513, 1418, 7776, 8081, 8081...   
3             304    1873  [5570, 5570, 7648, 8276, 1353, 1353, 1353, 135...   
4             312    1874  [940, 940, 2507, 4699, 6642, 6233, 6233, 227, ...   
...           ...     ...                                                ...   
57991     2494344  143847  [6432, 6432, 6432, 6432, 6432, 6432, 6432, 643...   
57992     2494390  143848  [5195, 5195, 829, 829, 831, 2599, 2599, 2599, ...   
57993     2494640  143856  [5905, 5905, 5905, 3178, 3178, 3178, 864, 866,...   
57994     2494653  143858  [7010, 7010, 7009, 7009, 7009, 3546, 2864, 780...   
57995     2494681  143859  [1146, 1146, 1146, 1146, 1146, 1146, 1146, 114...   

                                       

Pandas Apply:   0%|          | 0/57996 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/57996 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8620 [00:00<?, ?it/s]

In [41]:
# precalc adj matrices
adj = GTCModel.generate_node_traj_adj(network=network, traj_data=traj, k=1, bidirectional=False, add_self_loops=False)
np.savetxt("./traj_adj_k_1_for_temporal_tsd.gz", X=adj)

100%|██████████| 57996/57996 [00:26<00:00, 2162.47it/s]


In [4]:
# Pretrain tsd model 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
adj_t2v = np.loadtxt("./traj_adj_k_1_for_temporal_tsd.gz")
print(adj_t2v.shape)
data = network.generate_road_segment_pyg_dataset(only_edge_index=True)
traj2vec = Traj2VecModel(
            data,
            network,
            adj=adj_t2v,
            device=device,
            emb_dim=128,
            walk_length=30,
            context_size=5,
            walks_per_node=25,
            num_neg=10,
        )
traj2vec.train(epochs=30)

(8620, 8620)
Epoch: 1, avg_loss: 8.289206834400401
Epoch: 2, avg_loss: 5.029554535360897
Epoch: 3, avg_loss: 3.7701260535156025
Epoch: 4, avg_loss: 3.081674549947767
Epoch: 5, avg_loss: 2.6459378025111033
Epoch: 6, avg_loss: 2.3453161246051977
Epoch: 7, avg_loss: 2.12550759115139
Epoch: 8, avg_loss: 1.9578257440863287
Epoch: 9, avg_loss: 1.8257293034028383
Epoch: 10, avg_loss: 1.718999320268631
Epoch: 11, avg_loss: 1.6309666211273581
Epoch: 12, avg_loss: 1.5571258721836643
Epoch: 13, avg_loss: 1.4943010152330227
Epoch: 14, avg_loss: 1.4402011797583405
Epoch: 15, avg_loss: 1.3931278799678768
Epoch: 16, avg_loss: 1.3517971623908074
Epoch: 17, avg_loss: 1.3152203686200215
Epoch: 18, avg_loss: 1.2826225056367764
Epoch: 19, avg_loss: 1.2533872964396935
Epoch: 20, avg_loss: 1.227023006975651
Epoch: 21, avg_loss: 1.203123044608688
Epoch: 22, avg_loss: 1.1813626874258176
Epoch: 23, avg_loss: 1.1614641379517365
Epoch: 24, avg_loss: 1.1431990923250426
Epoch: 25, avg_loss: 1.1263754868507387
Epoc

In [5]:
traj2vec.save_model(path="")

In [3]:
# load gtc adj
adj = np.loadtxt("./traj_adj_k_2_bi_temporal.gz")

In [4]:
print(adj.shape)

(8620, 8620)


In [7]:
import gc 
del model
gc.collect()
torch.cuda.empty_cache()

In [4]:
# load tsd pre emb
data = network.generate_road_segment_pyg_dataset(only_edge_index=True)
tsd = Traj2VecModel(data, network, device=device)
tsd.load_model("model_tsd.pt")
tsd_emb = tsd.load_emb()

NameError: name 'device' is not defined

In [5]:
torch.cuda.empty_cache()
# load train data
data = torch.load("../../../datasets/trajectories/hanover/temporal/temporal_data.pt")
data = torch.swapaxes(data, 0, 1)
data = data.numpy()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
edge_index = network.generate_road_segment_pyg_dataset(only_edge_index=True).edge_index
# initialize model
model = TemporalGraphTrainer(data=data, adj=adj, edge_index=edge_index, struc_emb=None, device=device, batch_size=12)

(5952, 8620, 4)


In [6]:
model.load_model("model.pt")

In [7]:
# train model
model.train(epochs=10)

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/tgtc.py", line 133, in forward
    recon = self.decoder(z, X.shape[1])
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/tgtc.py", line 155, in forward
    x, _ = self._tdecoder(z)
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/modules/module.py", li

In [24]:
torch.save(model.model.state_dict(), os.path.join("model.pt"))

In [21]:
data.shape

(5952, 8620, 4)

In [22]:
# test sequence encoding 
z, _ = model.model(torch.Tensor(data[:12]).unsqueeze(0))
z = z.squeeze()
z.shape

torch.Size([8620, 128])

In [23]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# for m, e in models:
    # m.train(epochs=e)
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z.detach().cpu().numpy()] # gtc.load_emb(), gae_emb, rand_emb
for X in eva:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
    scorer = make_scorer(metrics.f1_score, average="macro")
    print(np.mean(cross_val_score(estimator=lm, X=X, y=y, scoring=scorer, cv=5)))

0.6420217717818654


# Test on traveltime task

In [ ]:
from evaluation.tasks.destination import DP_Dataset
from evaluation.tasks.next_location import NL_Dataset
from evaluation.tasks.travel_time import TTE_Dataset

from sklearn import model_selection

train, test = model_selection.train_test_split(
            traj_test, test_size=0.2, random_state=69
        )
train_loader = DataLoader(
    NL_Dataset(train, network),
    collate_fn=NL_Dataset.collate_fn_padd,
    batch_size=128,
    shuffle=True,
)
eval_loader = DataLoader(
    NL_Dataset(test, network),
    collate_fn=NL_Dataset.collate_fn_padd,
    batch_size=128,
)

In [ ]:
from operator import itemgetter

# Init 
class Test(nn.Module):
    def __init__(
        self,
        out_dim: int,
        device,
        plugin: nn.Module=None, 
        emb_dim: int = 128,
        hidden_units: int = 256,
        layers: int = 2,
        batch_size: int = 128,
    ):
        super(Test, self).__init__()
        self.encoder = nn.LSTM(
            emb_dim, hidden_units, num_layers=layers, batch_first=True, dropout=0.5
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_units, hidden_units * 2),
            nn.ReLU(),
            nn.Linear(hidden_units * 2, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, out_dim),
        )
        self.soft = nn.Softmax(dim=1)
        self.device = device
        self.loss = nn.CrossEntropyLoss()
        self.opt = torch.optim.Adam(self.parameters(), lr=0.001)
        self.drop = nn.Dropout(p=0.5)

        self.plugin = plugin
        self.encoder.to(device)
        self.decoder.to(device)

    def forward(self, x, lengths, neigh_masks, mode):
        batch_size, seq_len, _ = x.size()
        # encode static embedding
        x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True)

        x, _ = self.encoder(x)

        x, plengths = torch.nn.utils.rnn.pad_packed_sequence(
            x, batch_first=True, padding_value=0
        )
        x = x.contiguous()  # batch x seq x hidden
        # x = x.view(-1, x.shape[2])
        x = torch.stack(
            [x[b, plengths[b] - 1] for b in range(batch_size)]
        )  # get last valid item per batch batch x hidden
        
        if self.plugin is not None:
            x = self.plugin(x)

        yh = self.decoder(x)
        if mode == "train":
            yh = self.masked_out(yh, neigh_masks)

        return yh  # (batch x len(possible nodes))
    
    def predict(self, loader, emb_stat_full):
        with torch.no_grad():
            self.eval()
            yhs, ys = [], []
            for X, y, neigh_masks, lengths, mask, map in loader:
                # get static
                emb_static = self.get_embedding(emb_stat_full, X.clone(), mask, map)
                emb_static = emb_static.to(self.device)
                # get dynamic 
                if self.plugin is not None:
                    self.plugin.register_id_seq(X, mask, map, lengths)

                mask = mask.to(self.device)
    
                y = y.to(self.device)
                yh = self.soft(self.forward(emb_static, lengths.cpu().detach(), neigh_masks, "test")).argmax(dim=1)
                yhs.extend(yh.tolist())
                ys.extend(y.tolist())

            return np.array(yhs), np.array(ys)

    def train_model(self, loader, emb_stat_full, epochs=100):
        self.train()
        for e in tqdm(range(epochs)):
            total_loss = 0
            for X, y, neigh_masks, lengths, mask, map in tqdm(loader, leave=False):
                # get static
                emb_static = self.get_embedding(emb_stat_full, X.clone(), mask, map)
                emb_static = emb_static.to(self.device)
                # get dynamic 
                if self.plugin is not None:
                    self.plugin.register_id_seq(X, mask, map, lengths)
                mask = mask.to(device)
                # emb_batch = emb.activ1(emb.fc(traj_h))

                    # print(emb_batch.shape)
                y = y.to(self.device)
                yh = self.forward(emb_static, lengths.cpu().detach(), neigh_masks, "train")

                loss = self.loss(yh.squeeze(), y)
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
                total_loss += loss.item()

            print(f"Average training loss in episode {e}: {total_loss/len(loader)}")

    def get_embedding(self, emb, batch, mask, map):
        """
        Transform batch_size, seq_length, 1 to batch_size, seq_length, emb_size
        """
        res = torch.zeros((batch.shape[0], batch.shape[1], emb.shape[1]))
        for i, seq in enumerate(batch):
            emb_ids = itemgetter(*seq[mask[i]].tolist())(map)
            res[i, mask[i], :] = torch.Tensor(emb[emb_ids, :]).float()

        return res

    def masked_out(self, x, idxs):
        mask = torch.ones_like(x)
        for row, idx in zip(mask, idxs):
            row[idx] = 0
        x = x + (mask + 1e-44).log()
        # divider = torch.sum(torch.exp(x) * mask, axis=1).reshape(-1, 1)

        return x  # torch.log(torch.div(torch.exp(x) * mask, divider))

In [ ]:
task = Test(
            out_dim=len(
                network.line_graph.nodes
            ),
            plugin=plugin,
            device=device,
            emb_dim=model.model.transformer.embed.tok_embed.weight.shape[1],
            batch_size=128,
        )

task.train_model(loader=train_loader, emb_stat_full=model.load_emb(), epochs=10)